In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ba-reviews-uncleaned/BA_reviews_uncleaned.csv


In [9]:
df_reviews = pd.read_csv('/kaggle/input/ba-reviews-uncleaned/BA_reviews_uncleaned.csv')

In [10]:
df_reviews.head(2)

,Review,Overall_rating,Type Of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Date,verified
0,✅ Trip Verified | First time using BA busines...,9,Couple Leisure,Business Class,London to New York,5.0,5.0,5.0,5.0,NaN,4,yes,2024-01-07,True
1,Not Verified | Extremely rude ground service....,6,Family Leisure,Economy Class,Rome to London,4.0,5.0,4.0,1.0,NaN,2,no,2024-01-03,False


In [11]:

df_verified_reviews = df_reviews.loc[df_reviews['verified']== True]

df_verified_reviews.head(3)

,Review,Overall_rating,Type Of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Wifi & Connectivity,Value For Money,Recommended,Date,verified
0,✅ Trip Verified | First time using BA busines...,9,Couple Leisure,Business Class,London to New York,5.0,5.0,5.0,5.0,NaN,4,yes,2024-01-07,True
2,✅ Trip Verified | My son and I flew to Geneva...,1,Family Leisure,Business Class,Gatwick to Geneva,2.0,1.0,1.0,1.0,1.0,1,no,2024-01-02,True
3,✅ Trip Verified | For the price paid (bought ...,8,Solo Leisure,Business Class,Istanbul to London Heathrow,1.0,4.0,5.0,2.0,NaN,4,yes,2023-12-29,True


In [12]:
df_verified_reviews = df_verified_reviews.drop(['Type Of Traveller','Route','Seat Comfort', 'Cabin Staff Service', 'Food & Beverages',
       'Ground Service', 'Wifi & Connectivity', 'Value For Money','Date'],axis=1)


df_verified_reviews['Review'] = df_verified_reviews['Review'].str.split('|',expand=True)[1]

df_verified_reviews.head()


,Review,Overall_rating,Seat Type,Recommended,verified
0,First time using BA business class but we we...,9,Business Class,yes,True
2,My son and I flew to Geneva last Sunday for ...,1,Business Class,no,True
3,For the price paid (bought during a sale) it...,8,Business Class,yes,True
4,Flight left on time and arrived over half an...,6,Economy Class,no,True
5,"Very Poor Business class product, BA is not ...",2,Business Class,no,True


In [13]:
df_verified_reviews.reset_index(inplace=True)

AttributeError: 'NoneType' object has no attribute 'drop'

In [14]:
df_verified_reviews.drop('index',axis=1,inplace=True)

In [16]:


tokenizer = AutoTokenizer.from_pretrained("Swam80/test_trainer")
model = AutoModelForSequenceClassification.from_pretrained("Swam80/test_trainer")

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [17]:
def get_input_ids_and_attention_mask_chunk(review, tokenizer):
    """
    This function splits the input_ids and attention_mask into chunks of size 'chunksize'. 
    It also adds special tokens (101 for [CLS] and 102 for [SEP]) at the start and end of each chunk.
    If the length of a chunk is less than 'chunksize', it pads the chunk with zeros at the end.
    
    Args:
        review (str): The review text.
        tokenizer: The tokenizer.
    
    Returns:
        input_id_chunks (List[torch.Tensor]): List of chunked input_ids.
        attention_mask_chunks (List[torch.Tensor]): List of chunked attention_masks.
    """
    chunksize = 512
    tokens = tokenizer.encode_plus(review, add_special_tokens = False, return_tensors = 'pt')
    
    input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
    attention_mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))
    
    for i in range(len(input_id_chunks)):
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
        ])
        
        attention_mask_chunks[i] = torch.cat([
            torch.tensor([1]), attention_mask_chunks[i], torch.tensor([1])
        ])
        
        pad_length = chunksize - input_id_chunks[i].shape[0]
        
        if pad_length > 0:
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_length)
            ])
            attention_mask_chunks[i] = torch.cat([
                attention_mask_chunks[i], torch.Tensor([0] * pad_length)
            ])
            
    return input_id_chunks, attention_mask_chunks

In [20]:
df_verified_reviews.shape[0]

1132

In [21]:
# Initialize an empty dictionary to store the tokens
tokens_dict = {}

# Tokenize each review and store the tokens in the dictionary
for i in range(df_verified_reviews.shape[0]):
    tokens_dict[i] = get_input_ids_and_attention_mask_chunk(df_verified_reviews['Review'].iloc[i], tokenizer)


Token indices sequence length is longer than the specified maximum sequence length for this model (676 > 512). Running this sequence through the model will result in indexing errors


In [37]:
tokens_dict[0][0][0]

tensor([  101.,  2034.,  2051.,  2478.,  8670.,  2449.,  2465.,  2021.,  2057.,
         2020.,  7537.,  2007.,  1996.,  2326.,  2363.,  1012.,  2053.,  2028.,
         2001.,  3403.,  2012.,  4638.,  1999.,  1013.,  4524.,  4530.,  1998.,
         2083.,  3036.,  1999.,  1016.,  2781.,  1012.,  2057.,  2109.,  1996.,
        11549.,  1999.,  1996.,  1038.,  6733.,  2181.,  2029.,  2057.,  2179.,
         4251.,  2007.,  7564.,  1997.,  2833.,  1998.,  4392.,  2006.,  3749.,
         1012.,  9405.,  2001.,  4248.,  1998.,  1996.,  6644.,  2001.,  2006.,
         1037.,  1015.,  2595.,  2475.,  2595.,  2487.,  9621.,  2007.,  2260.,
         4272.,  2348.,  2045.,  2001.,  1037.,  3469.,  1038.,  1013.,  1039.,
         6644.,  2369.,  2008.,  2018.,  2055.,  2382.,  4272.,  1012.,  2153.,
         2833.,  1998.,  8974.,  2020.,  7564.,  2440.,  1998.,  2204.,  3737.,
         1012.,  2326.,  2013.,  1996.,  6644.,  3626.,  2001.,  6581.,  1998.,
         1996.,  6644.,  3208.,  1006., 

In [35]:
type(tokens_dict[0][0][0])

torch.Tensor

In [38]:
def process_review(key, tokens_dict, model):
    """
    This function takes a key to look up precomputed tokens (input_ids and attention_masks) in a dictionary,
    feeds them into the model, and returns the predicted sentiment for the review.
    
    Args:
        key (int): The key to look up the tokens in the dictionary.
        tokens_dict (dict): The dictionary of precomputed tokens.
        model: The sentiment analysis model.
    
    Returns:
        int: The predicted sentiment for the review.
    """
    # Look up the tokens in the dictionary
    tokens = tokens_dict[key]
    
    # Unpack the tokens
    input_id_chunks, attention_mask_chunks = tokens
    
    # Stack the chunks
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(attention_mask_chunks)
    
    # Prepare the inputs for the model
    input_dict = {
        'input_ids' : input_ids.long(),
        'attention_mask' : attention_mask.int()
    }
    
    # Get the model's output
    outputs = model(**input_dict)
    
    # Calculate the probabilities
    probabilities = torch.nn.functional.softmax(outputs[0], dim = -1 )
    
    # Calculate the mean probabilities
    mean_probabilities = probabilities.mean(dim = 0)
    
    # Return the index of the class with the highest mean probability
    return torch.argmax(mean_probabilities).item()

In [39]:
df_verified_reviews.index.to_series()

0          0
1          1
2          2
3          3
4          4
        ... 
1127    1127
1128    1128
1129    1129
1130    1130
1131    1131
Length: 1132, dtype: int64

In [40]:
df_verified_reviews['sentiment_value'] = df_verified_reviews.index.to_series().apply(lambda x: process_review(x, tokens_dict, model))

In [42]:
df_verified_reviews['sentiment_value'].value_counts()

sentiment_value
0    815
1    317
Name: count, dtype: int64

In [43]:
df_verified_reviews

,Review,Overall_rating,Seat Type,Recommended,verified,sentiment_value
0,First time using BA business class but we we...,9,Business Class,yes,True,1
1,My son and I flew to Geneva last Sunday for ...,1,Business Class,no,True,0
2,For the price paid (bought during a sale) it...,8,Business Class,yes,True,0
3,Flight left on time and arrived over half an...,6,Economy Class,no,True,0
4,"Very Poor Business class product, BA is not ...",2,Business Class,no,True,0
...,...,...,...,...,...,...
1127,London to Tel Aviv. I have been waiting on ...,1,Business Class,no,True,0
1128,I flew Premium Economy from London to Cape To...,3,Premium Economy,no,True,0
1129,Hong Kong to London before Christmas in Firs...,9,First Class,yes,True,1
1130,Dublin to Heathrow. BA gets worse and worse ...,1,Business Class,no,True,0


In [45]:
df_verified_reviews.to_csv('/kaggle/working/Sentiment_distilbert_chunks_verified_reviews1.csv')